# Coastal Grid

The Coastal Grid dataset provides a global tiling system for coastal analytics. It
supports scalable data processing workflows by offering coastal tiles at varying zoom 
levels (5, 6, 7, 8, 9, 10) and buffer sizes (500 m, 1000 m, 2000 m, 5000 m, 10000 m, 15000 m).

Each tile contains information on intersecting countries, continents, and Sentinel-2 MGRS tiles
as nested JSON lists. The dataset is particularly suited for applications requiring global coastal
coverage, such as satellite-based coastal monitoring, spatial analytics, and large-scale data processing.

This dataset is structured as a STAC collection, with individual items for each zoom level and buffer
size combination. Users can filter items by the `zoom` and `buffer_size` fields in the STAC metadata.

Please consider the following citation when using this dataset:

Floris Reinier Calkoen, Arjen Pieter Luijendijk, Kilian Vos, Etiënne Kras, Fedor Baart,
Enabling coastal analytics at planetary scale, Environmental Modelling & Software, 2024,
106257, ISSN 1364-8152, https://doi.org/10.1016/j.envsoft.2024.106257.

In [ ]:
import fsspec
import geopandas as gpd
import hvplot.pandas
import pystac

storage_options = {"account_name": "coclico"}

### Connect to the CoCliCo STAC 

In [ ]:
coclico_catalog = pystac.Catalog.from_file(
    "https://coclico.blob.core.windows.net/stac/v1/catalog.json"
)
coastal_grid_collection = coclico_catalog.get_child("coastal-grid")
coastal_grid_collection

### The collection includes multiple buffer distances

In [ ]:
list(coastal_grid_collection.get_all_items())

In [ ]:
item = coastal_grid_collection.get_item("coastal_grid_z7_10000m")
item

## The data is stored here

In [ ]:
href = item.assets["data"].href
href

## Read it into GeoPandas

In [ ]:
with fsspec.open(href, mode="rb", **storage_options) as f:
    coastal_grid = gpd.read_parquet(f)

In [ ]:
print(coastal_grid.shape)
coastal_grid.head()

## Show the data on an interactive map

In [ ]:
plot = coastal_grid.hvplot.polygons(
    geo=True,
    tiles="ESRI",
    color="coastal_grid:id",
    cmap="Category10",
    line_color="orange",
    alpha=0.8,
    width=800,
    legend=False,
)
plot